In [48]:
from IPython.core.display import HTML,Image
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm

In [51]:
import gc, argparse, sys, os, errno
from IPython.core.display import HTML,Image
from functools import reduce
import h5py
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook, show
output_notebook()
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn.neighbors import NearestNeighbors
from bokeh.palettes import Category20c
from ipywidgets import interact, FloatSlider,IntSlider, RadioButtons,Dropdown,Tab,Text

Populating the interactive namespace from numpy and matplotlib


/apps/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['indices', 'table']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Loading BokehJS ...

In [46]:
def embed_pdf_figure(width='640', height='480', title='Image'):
    data = BytesIO()
    plt.savefig(data, format='pdf', metadata={'Title': title})
    data = data.getvalue()
    data = 'data:application/pdf;base64,'+ str(b64encode(data), encoding='utf-8')
    display(HTML('<object width="{}" height="{}" data="{}" download="{}.pdf"></object>'.format(
        width, height, data, title)))
    plt.close()
    
from base64 import b64encode, b64decode
from io import StringIO, BytesIO

fonttitle = {'family':'Arial',
                  'weight' : 'normal', 
                  'size' : 8}
fontlabel = {'family':'Arial',
                  'weight' : 'normal', 
                  'size' : 6.5}
fontticklabel = {'family':'Arial',
                  'weight' : 'normal', 
                  'size' : 6.5}
fontlegend = {'family':'Arial',
                  'weight' : 'normal', 
              #'linewidth':0.5,
                  'size' : 6.5}
fontcbarlabel = {'family':'Arial',
                 'weight' : 'normal', 
                 #'Rotation' : 270,
                 #'labelpad' : 25,
                 'size' : 6.5}
fontcbarticklabel = {'family':'Arial',#Helvetica
                 'weight' : 'normal', 
                 'size' : 5.5}
def std_plot(ax,xlabel,ylabel,title=None,
             legendtitle=None,bbox_to_anchor=None,
             labelspacing=1.2,borderpad=0,handletextpad=0,legendsort=True,markerscale=None,
             xlim=None,ylim=None,
             xbins=None,ybins=None,
             cbar=None,cbarlabel=None,
             moveyaxis=False,sns=False,left=True):
    #plt.yticks([0,0.2,0.4,0.6,0.8,1.0],['0','0.2','0.4','0.6','0.8','1.0'])
    pyplot.draw()
    #plt.figure(linewidth=30.5)
    if xlim is not None:  
        ax.set(xlim=xlim)
    if ylim is not None:
        ax.set(ylim=ylim)
    #pyplot.draw()
    if xbins is not None:
        locator = MaxNLocator(nbins=xbins)
        locator.set_axis(ax.xaxis)
        ax.set_xticks(locator())
    if ybins is not None:
        locator = MaxNLocator(nbins=ybins)
        locator.set_axis(ax.yaxis)
        ax.set_yticks(locator())
    pyplot.draw()
    ax.set_xticks(ax.get_xticks())
    ax.set_yticks(ax.get_yticks())
    ax.set_xlabel(xlabel,fontdict = fontlabel,labelpad=5.5)
    ax.set_ylabel(ylabel,fontdict = fontlabel,labelpad=5.5)
    ax.set_xticklabels(ax.get_xticklabels(),fontticklabel)
    ax.set_yticklabels(ax.get_yticklabels(),fontticklabel)

    if moveyaxis is True:
        #fontticklabel 
        ax.spines['left'].set_position(('data',0))
    ax.spines['left'].set_visible(left)
    ax.spines['right'].set_visible(not left)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_linewidth(0.5)
    ax.spines['bottom'].set_linewidth(0.5)
    ax.spines['left'].set_linewidth(0.5)
    ax.spines['bottom'].set_color('k')
    ax.spines['left'].set_color('k')
    ax.spines['right'].set_color('k')
    
    ax.tick_params(direction='out', pad=2)
    #ax.spines['bottom']._edgecolor="#000000"
    #ax.spines['left']._edgecolor="#000000"
    if title is not None:
        ax.set_title(title,fontdict = fonttitle)
    if legendtitle is not None:
        #if legendloc is None:
        #    legendloc="best"
        legend = ax.legend(title=legendtitle,prop=fontlegend,
                      bbox_to_anchor=bbox_to_anchor,
                      labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                      edgecolor="#000000",fancybox=False,markerscale=markerscale)
        ax.legend_.get_frame()._linewidth=0.5
        legend.get_title().set_fontweight('normal')
        legend.get_title().set_fontsize(6.5)
        if legendsort is True:
            # h: handle l:label
            h,l = ax.get_legend_handles_labels()
            l,h = zip(*sorted(zip(l,h), key=lambda t: int(t[0]))) 
            legend = ax.legend(h,l,title=legendtitle,prop=fontlegend,
                      bbox_to_anchor=bbox_to_anchor,
                      labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                      edgecolor="#000000",fancybox=False,markerscale=markerscale)
            ax.legend_.get_frame()._linewidth=0.5
            legend.get_title().set_fontweight('normal')
            legend.get_title().set_fontsize(6.5)
        if sns is True:
            h,l = ax.get_legend_handles_labels()
            #l,h = zip(*sorted(zip(l,h), key=lambda t: int(t[0]))) 
            legend = ax.legend(h[1:],l[1:],title=legendtitle,prop=fontlegend,
                      bbox_to_anchor=bbox_to_anchor,
                      labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                      edgecolor="#000000",fancybox=False,markerscale=markerscale)
            ax.legend_.get_frame()._linewidth=0.5
            legend.get_title().set_fontweight('normal')
            legend.get_title().set_fontsize(6.5)

    if cbar is not None:
        #locator, formatter = cbar._get_ticker_locator_formatter()
        #ticks, ticklabels, offset_string = cbar._ticker(locator, formatter)
        #cbar.ax.spines['top'].set_visible(False)
        #cbar.ax.spines['right'].set_visible(False)
        #cbar.ax.spines['bottom'].set_visible(False)
        #cbar.ax.spines['left'].set_visible(False)
        cbar.ax.tick_params(direction='out', pad=3,width=0,length=0)
        cbar.set_label(cbarlabel,fontdict = fontcbarlabel,Rotation=270,labelpad=7.5)
        cbar.ax.set_yticks(cbar.ax.get_yticks())
        cbar.ax.set_yticklabels(cbar.ax.get_yticklabels(),fontcbarticklabel)
    return ax

def gradient_func(val):
    return '<span style="background: linear-gradient(90deg, #d65f5f {0}%, transparent 0%)">{0:.3f}</span>'.format(val)

def display_dataframe(df, filename=None, encoding='utf-8', format='csv', type='button',gradientfunc=False, **kwargs):
    #display(df)
    #if isinstance(df, pd.DataFrame):
    #    display(df.style.set_caption(filename))
    #else:
    if gradientfunc == False:
        display(df.style.set_caption(filename))    
    else:
        display(df.style.format(gradient_func).set_caption(filename)) 
    if filename is None:
        filename = "dataframe"
    if format == 'csv':
        data = df.to_csv(**kwargs)
        mime_type = 'text/csv'
        filename = filename + '.csv'
    elif format == 'tsv':
        data = df.to_csv(**kwargs)
        mime_type = 'text/plain'
        filename = filename + '.txt'
    else:
        raise ValueError('unknown file format: {}'.format(format))
    data = 'data:{mime_type};base64,'.format(mime_type=mime_type) + str(b64encode(bytes(data, encoding=encoding)), encoding=encoding)
    if type == 'hyperlink':
        display(HTML('<a href=" " download={filename} target="_blank">{filename}</a >'.format(
            mime_type=mime_type, filename=filename, data=data)))
    elif type == 'button':
        button_id = 'button_{}'.format(np.random.randint(1000000000))
        display(HTML(r'<input type="button" id="{0}" value="Download">'.format(button_id)))
        display(HTML('''<script>
    document.getElementById("{button_id}").addEventListener("click", function(event){{
        var filename = "{filename}";
        var data = "{data}";
        const element = document.createElement('a');
        element.setAttribute('href', data);
        element.setAttribute('download', filename);
        element.style.display = 'none';
        document.body.appendChild(element);
        element.click();
        document.body.removeChild(element);
    }});
</script>'''.format(button_id=button_id, filename=filename, data=data)))

In [2]:
mat1 = pd.read_table('/home/xieyufeng/ex/output/scirep/count_matrix/mirna_and_domains.txt',index_col=0)
mat2 = pd.read_table('/Share/home/shibinbin/projects/exSeek-dev/output/cfRNA/count_matrix/mirna_and_domains.txt',index_col=0)
#pd.concat([mat1, mat2], axis=1, join='inner')
class_info2 = pd.read_table('/Share/home/shibinbin/projects/exSeek-dev/data/cfRNA/sample_classes.txt',index_col=0)
class_info1 = pd.read_table('/home/xieyufeng/ex/data/scirep/sample_classes.no_stage.txt',index_col=0)



In [58]:
batch_info = pd.read_table('/home/xieyufeng/ex/data/scirep/batch_info.txt',index_col=0)
batch_info

,RNA Isolation batch,library prepration day,gel cut size selection
Sample_1S1,2,22,7
Sample_1S2,2,22,8
Sample_1S3,2,22,1
Sample_2S1,2,22,2
Sample_2S2,2,22,3
Sample_2S3,2,22,4
Sample_3S1,2,22,5
Sample_3S2,2,22,6
Sample_3S3,2,22,7
Sample_4S1,2,22,8


In [22]:
def knn_score(X, y, K=10):
    N = X.shape[0]
    assert K < N
    nn = NearestNeighbors(K)
    nn.fit(X)
    distances, indices = nn.kneighbors(X, K + 1)
    neighbor_classes = np.take(y, indices[:, 1:])
    same_class_fractions = np.sum(neighbor_classes == y[:, np.newaxis], axis=1)
    classes, counts = np.unique(y, return_counts=True)
    classes = np.argmax(y.reshape((-1, 1)) == classes.reshape((1, -1)), axis=1)
    counts = np.take(counts, classes)
    mean_r = K/(N - 1)*counts
    max_r = np.minimum(K, counts)
    #print (same_class_fractions.shape,mean_r.shape,max_r.shape)
    #scores = (np.mean(same_class_fractions) - mean_r)/(max_r - mean_r)
    scores = (same_class_fractions - mean_r)/(max_r - mean_r)
    #print(scores)
    return scores.mean()

In [23]:
def get_knn_score(data,sampleclass,method_PCA = True,prediction_algorithm='knn'):
    X = np.log2(data + 0.001).T
    X = StandardScaler().fit_transform(X)
    if method_PCA == True:
        transform = PCA()
    else:
        transform = TSNE()
    X_pca = transform.fit_transform(X)
    X_, y_ = X_pca, sampleclass.loc[data.columns.values].values.ravel() 
    knn_score_ = knn_score(X_, y_)
    return knn_score_

In [91]:
def uca_scores(X,y, prediction_algorithm='knn'):
    from sklearn.metrics import adjusted_rand_score as ARI
    from sklearn.metrics import normalized_mutual_info_score as NMI
    from sklearn.metrics import silhouette_score
    from sklearn.mixture import GaussianMixture as GMM
    from sklearn.cluster import KMeans

    cluster_num = np.unique(y).shape[0]
    if prediction_algorithm == 'knn':
        labels_pred = KMeans(cluster_num, n_init=200).fit_predict(X)  
    elif prediction_algorithm == 'gmm':
        gmm = GMM(cluster_num)
        gmm.fit(X)
        labels_pred = gmm.predict(X)
    labels = y
    #asw_score = silhouette_score(X, labels)
    #nmi_score = NMI(labels, labels_pred)
    #ari_score = ARI(labels, labels_pred)
    labels_int = convert_label_to_int(labels)
    uca_score = unsupervised_clustering_accuracy(labels_int, labels_pred)[0]
    return uca_score

In [105]:
def convert_label_to_int(sample_class):
    classes, counts = np.unique(sample_class, return_counts=True)
    classes = np.argmax(sample_class.reshape((-1, 1)) == classes.reshape((1, -1)), axis=1)
    return classes

def unsupervised_clustering_accuracy(y, y_pred):
    from sklearn.utils.linear_assignment_ import linear_assignment
    assert len(y_pred) == len(y)
    u = np.unique(np.concatenate((y, y_pred)))
    n_clusters = len(u)
    mapping = dict(zip(u, range(n_clusters)))
    reward_matrix = np.zeros((n_clusters, n_clusters), dtype=np.int64)
    for y_pred_, y_ in zip(y_pred, y):
        if y_ in mapping:
            reward_matrix[mapping[y_pred_], mapping[y_]] += 1
    cost_matrix = reward_matrix.max() - reward_matrix
    ind = linear_assignment(cost_matrix)
    return sum([reward_matrix[i, j] for i, j in ind]) * 1.0 / y_pred.size, ind

In [92]:
def get_uca_score(data,sampleclass,method_PCA = True,prediction_algorithm='knn'):
    X = np.log2(data + 0.001).T
    X = StandardScaler().fit_transform(X)
    if method_PCA == True:
        transform = PCA()
    else:
        transform = TSNE()
    X_pca = transform.fit_transform(X)
    X_, y_ = X_pca, sampleclass.loc[data.columns.values].values.ravel() 
    uca_score = uca_scores(X_, y_,prediction_algorithm)
    return uca_score

# batch 0

In [66]:
sampleclass = batch_info.iloc[:,0]

In [67]:
cd ~/ex

/home/xieyufeng/ex


In [68]:
uca_summary = pd.read_table('output/'+'scirep'+'/select_preprocess_method/uca_score/'+'domains_combined'+'/summary.txt',
                           index_col=0)
knn_list=[]
for i in tqdm(uca_summary.index):
    table = pd.read_table('~/ex/output/'+'scirep'+'/matrix_processing/'+i+'.domains_combined.txt',
                           index_col=0)
    knn_list.append(get_knn_score(table,sampleclass))
    

HBox(children=(IntProgress(value=0, max=75), HTML(value='')))

In [69]:
knn_summary = pd.DataFrame(data={'preprocess_method':uca_summary.index,'knn_score':list(knn_list)})
knn_summary = knn_summary.set_index('preprocess_method')

In [70]:
knn_summary_table = pd.DataFrame(np.ndarray([knn_summary.shape[0],4]))
knn_summary_table.columns = ['imputation','normalization','batch_removal','knn_score']
for i in range(knn_summary.shape[0]):
    splittmp = knn_summary.index[i].split('.')
    knn_summary_table.iloc[i,0] = splittmp[1]
    knn_summary_table.iloc[i,1] = splittmp[2]
    knn_summary_table.iloc[i,2] = splittmp[3]
    knn_summary_table.iloc[i,3] = knn_summary.values[i]
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[0]).mean(),filename='Imputation Methods knn Score',gradientfunc=False)
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[1]).mean(),filename='Normalization Methods knn Score',gradientfunc=False)
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[2]).mean(),filename='Batch Removal Methods knn Score',gradientfunc=False)
knn_mean_summary = pd.concat((knn_summary_table.groupby(knn_summary_table.columns[0]).mean(),
          knn_summary_table.groupby(knn_summary_table.columns[1]).mean(),
          knn_summary_table.groupby(knn_summary_table.columns[2]).mean()))
knn_mean_summary =  knn_mean_summary.rename(index={'null': 'imputation_null'})

,knn_score
imputation,
null,0.279079
scimpute_count,0.280723
viper_count,0.248548


,knn_score
normalization,
Norm_CPM,0.267633
Norm_CPM_top,0.264693
Norm_RLE,0.273684
Norm_SCnorm,0.279327
Norm_TMM,0.261914


,knn_score
batch_removal,
Batch_Combat_1,0.0676102
Batch_Combat_2,0.227371
Batch_Combat_3,0.411901
Batch_RUV,0.217858
Batch_null,0.422511


In [71]:
fig, ax = plt.subplots(1,figsize=(7, 4))
knn_mean_summary.plot(kind='bar',ax=ax,width=0.8,
                legend=True)
std_plot(ax,'','')
legend = ax.legend(prop=fontlegend,
              bbox_to_anchor=[0.2,1],
              borderpad=1,
              edgecolor="#000000",fancybox=False)
ax.legend_.get_frame()._linewidth=0.5
legend.get_title().set_fontweight('normal')
legend.get_title().set_fontsize(6.5)
fig.tight_layout()

embed_pdf_figure()
#fig.tight_layout()
#plt.figure(linewidth=30.5)
#ax.spines['right'].set_visible(False)
#ax.spines['top'].set_visible(False)

In [72]:
merge = pd.concat([knn_summary,uca_summary],axis=1)
merge['impute'] = [method.split('.')[1] for method in merge.index]
merge['normalization'] = [method.split('.')[2] for method in merge.index]
merge['batch'] = [method.split('.')[3] for method in merge.index]
sizelist=[10,50,200]
impute_list = np.unique(merge['impute'])
merge['imputation_size'] = merge['impute']
for i in np.arange(len(impute_list)):
    where = np.where(merge['imputation_size']==impute_list[i])
    for j in where:
        merge['imputation_size'].iloc[j]=sizelist[i]

In [74]:
fig,ax=plt.subplots(figsize=(7,5))
ax = sns.scatterplot(x='uca_score',y='knn_score',data = merge,hue='batch',style='normalization',size='impute')
std_plot(ax,'UCA score','kNN score','method batch visualization')
legend = ax.legend(prop=fontlegend,
                  #labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                  edgecolor="#000000",fancybox=False)
ax.legend_.get_frame()._linewidth=0.5
legend.get_title().set_fontweight('normal')
legend.get_title().set_fontsize(6.5)

fig.tight_layout()
embed_pdf_figure()

# batch 1

In [75]:
sampleclass = batch_info.iloc[:,1]

In [76]:
cd ~/ex

/home/xieyufeng/ex


In [77]:
uca_summary = pd.read_table('output/'+'scirep'+'/select_preprocess_method/uca_score/'+'domains_combined'+'/summary.txt',
                           index_col=0)
knn_list=[]
for i in tqdm(uca_summary.index):
    table = pd.read_table('~/ex/output/'+'scirep'+'/matrix_processing/'+i+'.domains_combined.txt',
                           index_col=0)
    knn_list.append(get_knn_score(table,sampleclass))
    

HBox(children=(IntProgress(value=0, max=75), HTML(value='')))

In [78]:
knn_summary = pd.DataFrame(data={'preprocess_method':uca_summary.index,'knn_score':list(knn_list)})
knn_summary = knn_summary.set_index('preprocess_method')

In [79]:
knn_summary_table = pd.DataFrame(np.ndarray([knn_summary.shape[0],4]))
knn_summary_table.columns = ['imputation','normalization','batch_removal','knn_score']
for i in range(knn_summary.shape[0]):
    splittmp = knn_summary.index[i].split('.')
    knn_summary_table.iloc[i,0] = splittmp[1]
    knn_summary_table.iloc[i,1] = splittmp[2]
    knn_summary_table.iloc[i,2] = splittmp[3]
    knn_summary_table.iloc[i,3] = knn_summary.values[i]
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[0]).mean(),filename='Imputation Methods knn Score',gradientfunc=False)
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[1]).mean(),filename='Normalization Methods knn Score',gradientfunc=False)
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[2]).mean(),filename='Batch Removal Methods knn Score',gradientfunc=False)
knn_mean_summary = pd.concat((knn_summary_table.groupby(knn_summary_table.columns[0]).mean(),
          knn_summary_table.groupby(knn_summary_table.columns[1]).mean(),
          knn_summary_table.groupby(knn_summary_table.columns[2]).mean()))
knn_mean_summary =  knn_mean_summary.rename(index={'null': 'imputation_null'})

,knn_score
imputation,
null,0.228718
scimpute_count,0.232035
viper_count,0.192154


,knn_score
normalization,
Norm_CPM,0.214303
Norm_CPM_top,0.216023
Norm_RLE,0.223377
Norm_SCnorm,0.221737
Norm_TMM,0.212739


,knn_score
batch_removal,
Batch_Combat_1,0.090888
Batch_Combat_2,0.16599
Batch_Combat_3,0.34479
Batch_RUV,0.149175
Batch_null,0.337336


In [80]:
fig, ax = plt.subplots(1,figsize=(7, 4))
knn_mean_summary.plot(kind='bar',ax=ax,width=0.8,
                legend=True)
std_plot(ax,'','')
legend = ax.legend(prop=fontlegend,
              bbox_to_anchor=[0.2,1],
              borderpad=1,
              edgecolor="#000000",fancybox=False)
ax.legend_.get_frame()._linewidth=0.5
legend.get_title().set_fontweight('normal')
legend.get_title().set_fontsize(6.5)
fig.tight_layout()

embed_pdf_figure()
#fig.tight_layout()
#plt.figure(linewidth=30.5)
#ax.spines['right'].set_visible(False)
#ax.spines['top'].set_visible(False)

In [81]:
merge = pd.concat([knn_summary,uca_summary],axis=1)
merge['impute'] = [method.split('.')[1] for method in merge.index]
merge['normalization'] = [method.split('.')[2] for method in merge.index]
merge['batch'] = [method.split('.')[3] for method in merge.index]
sizelist=[10,50,200]
impute_list = np.unique(merge['impute'])
merge['imputation_size'] = merge['impute']
for i in np.arange(len(impute_list)):
    where = np.where(merge['imputation_size']==impute_list[i])
    for j in where:
        merge['imputation_size'].iloc[j]=sizelist[i]

In [82]:
fig,ax=plt.subplots(figsize=(7,5))
ax = sns.scatterplot(x='uca_score',y='knn_score',data = merge,hue='batch',style='normalization',size='impute')
std_plot(ax,'UCA score','kNN score','method batch visualization')
legend = ax.legend(prop=fontlegend,
                  #labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                  edgecolor="#000000",fancybox=False)
ax.legend_.get_frame()._linewidth=0.5
legend.get_title().set_fontweight('normal')
legend.get_title().set_fontsize(6.5)

fig.tight_layout()
embed_pdf_figure()

# batch 2

In [83]:
sampleclass = batch_info.iloc[:,2]

In [84]:
cd ~/ex

/home/xieyufeng/ex


In [85]:
uca_summary = pd.read_table('output/'+'scirep'+'/select_preprocess_method/uca_score/'+'domains_combined'+'/summary.txt',
                           index_col=0)
knn_list=[]
for i in tqdm(uca_summary.index):
    table = pd.read_table('~/ex/output/'+'scirep'+'/matrix_processing/'+i+'.domains_combined.txt',
                           index_col=0)
    knn_list.append(get_knn_score(table,sampleclass))
    

HBox(children=(IntProgress(value=0, max=75), HTML(value='')))

In [86]:
knn_summary = pd.DataFrame(data={'preprocess_method':uca_summary.index,'knn_score':list(knn_list)})
knn_summary = knn_summary.set_index('preprocess_method')

In [87]:
knn_summary_table = pd.DataFrame(np.ndarray([knn_summary.shape[0],4]))
knn_summary_table.columns = ['imputation','normalization','batch_removal','knn_score']
for i in range(knn_summary.shape[0]):
    splittmp = knn_summary.index[i].split('.')
    knn_summary_table.iloc[i,0] = splittmp[1]
    knn_summary_table.iloc[i,1] = splittmp[2]
    knn_summary_table.iloc[i,2] = splittmp[3]
    knn_summary_table.iloc[i,3] = knn_summary.values[i]
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[0]).mean(),filename='Imputation Methods knn Score',gradientfunc=False)
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[1]).mean(),filename='Normalization Methods knn Score',gradientfunc=False)
display_dataframe(knn_summary_table.groupby(knn_summary_table.columns[2]).mean(),filename='Batch Removal Methods knn Score',gradientfunc=False)
knn_mean_summary = pd.concat((knn_summary_table.groupby(knn_summary_table.columns[0]).mean(),
          knn_summary_table.groupby(knn_summary_table.columns[1]).mean(),
          knn_summary_table.groupby(knn_summary_table.columns[2]).mean()))
knn_mean_summary =  knn_mean_summary.rename(index={'null': 'imputation_null'})

,knn_score
imputation,
null,0.00976294
scimpute_count,0.00910413
viper_count,0.0119819


,knn_score
normalization,
Norm_CPM,0.0119364
Norm_CPM_top,0.0142308
Norm_RLE,0.00835493
Norm_SCnorm,0.00978494
Norm_TMM,0.00710796


,knn_score
batch_removal,
Batch_Combat_1,0.00572827
Batch_Combat_2,0.0124363
Batch_Combat_3,-0.015056
Batch_RUV,0.0308668
Batch_null,0.0174397


In [88]:
fig, ax = plt.subplots(1,figsize=(7, 4))
knn_mean_summary.plot(kind='bar',ax=ax,width=0.8,
                legend=True)
std_plot(ax,'','')
legend = ax.legend(prop=fontlegend,
              bbox_to_anchor=[0.2,1],
              borderpad=1,
              edgecolor="#000000",fancybox=False)
ax.legend_.get_frame()._linewidth=0.5
legend.get_title().set_fontweight('normal')
legend.get_title().set_fontsize(6.5)
fig.tight_layout()

embed_pdf_figure()
#fig.tight_layout()
#plt.figure(linewidth=30.5)
#ax.spines['right'].set_visible(False)
#ax.spines['top'].set_visible(False)

In [89]:
merge = pd.concat([knn_summary,uca_summary],axis=1)
merge['impute'] = [method.split('.')[1] for method in merge.index]
merge['normalization'] = [method.split('.')[2] for method in merge.index]
merge['batch'] = [method.split('.')[3] for method in merge.index]
sizelist=[10,50,200]
impute_list = np.unique(merge['impute'])
merge['imputation_size'] = merge['impute']
for i in np.arange(len(impute_list)):
    where = np.where(merge['imputation_size']==impute_list[i])
    for j in where:
        merge['imputation_size'].iloc[j]=sizelist[i]

In [90]:
fig,ax=plt.subplots(figsize=(7,5))
ax = sns.scatterplot(x='uca_score',y='knn_score',data = merge,hue='batch',style='normalization',size='impute')
std_plot(ax,'UCA score','kNN score','method batch visualization')
legend = ax.legend(prop=fontlegend,
                  #labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                  edgecolor="#000000",fancybox=False)
ax.legend_.get_frame()._linewidth=0.5
legend.get_title().set_fontweight('normal')
legend.get_title().set_fontsize(6.5)

fig.tight_layout()
embed_pdf_figure()

# merge knn-uca

In [93]:
normlist= ['filter.null','filter.null.Norm_CPM',#'filter.null.Norm_CPM_rm',
           'filter.null.Norm_CPM_top_5','filter.null.Norm_CPM_top_10',
           'filter.null.Norm_CPM_top_20','filter.null.Norm_CPM_top_40',
           'filter.null.Norm_RLE','filter.null.Norm_TMM','filter.null.Norm_UQ']

In [110]:
batch_info = pd.read_table('/home/xieyufeng/fig3/data/merge/batch_info.txt',index_col=1)
class_info = pd.read_table('/home/xieyufeng/fig3/data/merge/sample_classes.txt',index_col=0)

In [106]:
sampleclass = class_info
uca_list=[]
for i in tqdm(normlist):
    table = pd.read_table('/home/xieyufeng/fig3/output/'+'merge'+'/matrix_processing/'+i+'.mirna_and_domains.txt',
                           index_col=0)
    uca_list.append(get_uca_score(table,sampleclass))

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

In [108]:
uca_summary = pd.DataFrame(data={'preprocess_method':normlist,'uca_score':list(uca_list)})
uca_summary = uca_summary.set_index('preprocess_method')

In [113]:
sampleclass = batch_info.iloc[:,0]
knn_list=[]
for i in tqdm(normlist):
    table = pd.read_table('/home/xieyufeng/fig3/output/'+'merge'+'/matrix_processing/'+i+'.mirna_and_domains.txt',
                           index_col=0)
    knn_list.append(get_knn_score(table,sampleclass))

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

In [114]:
knn_summary = pd.DataFrame(data={'preprocess_method':uca_summary.index,'knn_score':list(knn_list)})
knn_summary = knn_summary.set_index('preprocess_method')

In [116]:
merge.index

Index(['filter.null', 'filter.null.Norm_CPM', 'filter.null.Norm_CPM_top_5',
       'filter.null.Norm_CPM_top_10', 'filter.null.Norm_CPM_top_20',
       'filter.null.Norm_CPM_top_40', 'filter.null.Norm_RLE',
       'filter.null.Norm_TMM', 'filter.null.Norm_UQ'],
      dtype='object', name='preprocess_method')

In [117]:
merge = pd.concat([knn_summary,uca_summary],axis=1)
merge = merge.drop(['filter.null'])
merge['normalization'] = [method.split('.')[2] for method in merge.index]
fig,ax=plt.subplots(figsize=(7,5))
ax = sns.scatterplot(x='uca_score',y='knn_score',data = merge,hue='normalization')
std_plot(ax,'UCA score','kNN score','method batch visualization')
legend = ax.legend(prop=fontlegend,
                  #labelspacing=labelspacing,borderpad=borderpad,handletextpad=handletextpad,
                  edgecolor="#000000",fancybox=False)
ax.legend_.get_frame()._linewidth=0.5
legend.get_title().set_fontweight('normal')
legend.get_title().set_fontsize(6.5)

fig.tight_layout()
embed_pdf_figure()